In [78]:
import numpy as np
import pandas as pd
from pandas import DataFrame, read_csv, to_numeric
from sklearn import cluster
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import math

Q1: Parametric Classification: Using each of the 64 input features separately as the single input dimension, use parametric classification, assuming that the input is distributed according to a Gaussian. Report the training and test errors for the case of each of the 64 features. Which feature(s) give the best test performance? 

In [79]:

train_data = read_csv('/media/anuja/study/Anuja/MS/CS_697A_ML/hw2/optdigits_69_N200.tra', sep=",")
train_data

,0,0.1,5,14,4,0.2,0.3,0.4,0.5,0.6,...,0.27,0.28,0.29,4.6,12.1,14.4,7,0.30,0.31,6.1
0,0,0,0,10,12,0,0,0,0,0,...,0,0,0,1,11,14,12,1,0,6
1,0,0,2,14,10,0,0,0,0,1,...,0,0,0,1,12,14,12,4,0,6
2,0,0,2,13,5,0,0,0,0,0,...,0,0,0,4,15,12,9,1,0,6
3,0,0,2,13,6,0,0,0,0,0,...,0,0,0,2,12,14,12,4,0,6
4,0,0,2,13,6,0,0,0,0,0,...,0,0,0,2,12,14,12,4,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,0,0,8,13,10,5,0,0,0,...,0,0,0,0,14,15,2,0,0,9
196,0,0,0,3,10,13,12,4,0,0,...,0,0,0,0,1,15,1,0,0,9
197,0,0,0,4,10,13,12,0,0,0,...,0,0,0,0,2,11,0,0,0,9
198,0,0,6,14,16,9,0,0,0,5,...,0,0,0,8,14,16,13,1,0,9


In [80]:
# use train_array as numpy array

train_array = np.asarray(train_data)
print(train_array)
[train_rows, train_col] = train_array.shape

# now convert split the arrays into two parts as we have two classes here
train_class6 = train_array[train_array[:,64] == 6, :]
print(train_class6)

train_class9 = train_array[train_array[:,64] == 9, :]
print(train_class9)

[[0 0 0 ... 1 0 6]
 [0 0 2 ... 4 0 6]
 [0 0 2 ... 1 0 6]
 ...
 [0 0 0 ... 0 0 9]
 [0 0 6 ... 1 0 9]
 [0 0 2 ... 4 0 9]]
[[0 0 0 ... 1 0 6]
 [0 0 2 ... 4 0 6]
 [0 0 2 ... 1 0 6]
 ...
 [0 0 0 ... 7 0 6]
 [0 0 2 ... 4 0 6]
 [0 0 1 ... 3 0 6]]
[[0 0 0 ... 0 0 9]
 [0 0 1 ... 0 0 9]
 [0 0 0 ... 0 0 9]
 ...
 [0 0 0 ... 0 0 9]
 [0 0 6 ... 1 0 9]
 [0 0 2 ... 4 0 9]]


In [81]:
# get the occurances or count of both the classes

[train_class6_rows, train_col] = train_class6.shape
print("Number of rows of class 6 of training dataset : ", train_class6_rows)

[train_class9_rows, train_col] = train_class9.shape
print("Number of rows of class 9 of training dataset : ",train_class9_rows)


Number of rows of class 6 of training dataset :  100
Number of rows of class 9 of training dataset :  100


In [82]:
# now will fine out the class priors

train_prior_class6 = train_class6_rows/train_rows
print("Prior probability of class 6 of training data : ", train_prior_class6)

train_prior_class9 = train_class9_rows/train_rows
print("Prior probability of class 9 of training data : ", train_prior_class9)


Prior probability of class 6 of training data :  0.5
Prior probability of class 9 of training data :  0.5


In [83]:
# now find the mean and std deviation for each feature of the classes (6 and 9). We have 64 features here and 2 classes.

train_df = pd.DataFrame(columns = ["Train_Mean_class6", "Train_Std_Deviation_class6", "Train_Mean_class9", "Train_Std_Deviation_class9"])

for icol in range(train_col-1):
    train_mean_class6 = np.mean(train_class6[:,icol])
    train_std_dv_class6 = np.std(train_class6[:,icol])

    train_mean_class9 = np.mean(train_class9[:,icol])
    train_std_dv_class9 = np.std(train_class9[:,icol])
    
    train_df = train_df.append({'Train_Mean_class6': train_mean_class6, 'Train_Std_Deviation_class6': train_std_dv_class6, 'Train_Mean_class9': train_mean_class9, 'Train_Std_Deviation_class9': train_std_dv_class9}, ignore_index=True)


print(train_df)



    Train_Mean_class6  Train_Std_Deviation_class6  Train_Mean_class9  \
0                0.00                    0.000000               0.00   
1                0.00                    0.000000               0.10   
2                1.59                    2.898603               4.32   
3               10.38                    3.405231              10.99   
4                8.19                    5.266299              11.44   
..                ...                         ...                ...   
59              10.01                    2.988294              11.35   
60              14.62                    1.369525              12.42   
61              13.60                    2.619160               7.45   
62               6.50                    4.332436               1.61   
63               0.29                    1.070467               0.05   

    Train_Std_Deviation_class9  
0                     0.000000  
1                     0.331662  
2                     3.627341  
3  

In [99]:

# for class 6 find error of each feature

# train_feature_errors_class6 = []
train_feature_errors_class6_dict = {}

for icol in range(train_col-1):
    train_mean_class6 = train_df.Train_Mean_class6[icol]
    train_std_class6 = train_df.Train_Std_Deviation_class6[icol]
    train_mean_class9 = train_df.Train_Mean_class9[icol]
    train_std_class9 = train_df.Train_Std_Deviation_class9[icol]
    train_error_of_feature = 0
    for irow in range (train_class6_rows):
        if (~(train_std_class6 == 0.0) & ~(train_std_class9 == 0.0)):
            train_g_class6 = -((1/2)* math.log(2*math.pi)) - math.log(train_std_class6) - ((train_class6[irow,icol] - train_mean_class6)** 2)/(2*(train_std_class6**2)) + math.log(train_prior_class6)
            train_g_class9 = -((1/2)* math.log(2*math.pi)) - math.log(train_std_class9) - ((train_class6[irow,icol] - train_mean_class9)** 2)/(2*(train_std_class9**2)) + math.log(train_prior_class6)
            if (train_g_class6 > train_g_class9):
                if ~(train_class6[irow, icol] == 6):
                    train_error_of_feature = train_error_of_feature + 1
            elif (train_g_class6 < train_g_class9):
                if ~(train_class6[irow, icol] == 9):
                    train_error_of_feature = train_error_of_feature + 1
    train_feature_errors_class6_dict.update({"feature" + str(icol+1) :train_error_of_feature })
    


# for class 9 find error of each feature

for icol in range(train_col-1):
    train_mean_class6 = train_df.Train_Mean_class6[icol]
    train_std_class6 = train_df.Train_Std_Deviation_class6[icol]
    train_mean_class9 = train_df.Train_Mean_class9[icol]
    train_std_class9 = train_df.Train_Std_Deviation_class9[icol]
    train_error_of_feature = 0
    for irow in range (train_class9_rows):
        if (~(train_std_class6 == 0.0) & ~(train_std_class9 == 0.0)):
            train_g_class6 = -((1/2)* math.log(2*math.pi)) - math.log(train_std_class6) - ((train_class6[irow,icol] - train_mean_class6)** 2)/(2*(train_std_class6**2)) + math.log(train_prior_class6)
            train_g_class9 = -((1/2)* math.log(2*math.pi)) - math.log(train_std_class9) - ((train_class9[irow,icol] - train_mean_class9)** 2)/(2*(train_std_class9**2)) + math.log(train_prior_class6)
            if (train_g_class6 > train_g_class9):
                if ~(train_class6[irow, icol] == 6):
                    train_error_of_feature = train_error_of_feature + 1
            elif (train_g_class6 < train_g_class9):
                if ~(train_class6[irow, icol] == 9):
                    train_error_of_feature = train_error_of_feature + 1
            
    
    #get the dictionary value here for current feature and update it
    train_feature_errors_class6_dict["feature" + str(icol+1)]=  train_error_of_feature + train_feature_errors_class6_dict["feature" + str(icol+1)]

    
print("-------------printing list of training errors for each feature-----------------" + " \n")    
print(train_feature_errors_class6_dict)
print("\n")
print("sorted : ")
train_feature_errors_class6_sortedDict = sorted(train_feature_errors_class6_dict.items(), key=lambda item: item[1], reverse = False)
print(train_feature_errors_class6_sortedDict)



-------------printing list of training errors for each feature----------------- 

{'feature1': 0, 'feature2': 0, 'feature3': 196, 'feature4': 195, 'feature5': 185, 'feature6': 200, 'feature7': 0, 'feature8': 0, 'feature9': 0, 'feature10': 200, 'feature11': 186, 'feature12': 194, 'feature13': 186, 'feature14': 200, 'feature15': 0, 'feature16': 0, 'feature17': 0, 'feature18': 200, 'feature19': 193, 'feature20': 190, 'feature21': 200, 'feature22': 0, 'feature23': 0, 'feature24': 0, 'feature25': 0, 'feature26': 193, 'feature27': 200, 'feature28': 177, 'feature29': 191, 'feature30': 196, 'feature31': 200, 'feature32': 0, 'feature33': 0, 'feature34': 197, 'feature35': 198, 'feature36': 200, 'feature37': 200, 'feature38': 193, 'feature39': 196, 'feature40': 0, 'feature41': 0, 'feature42': 196, 'feature43': 199, 'feature44': 191, 'feature45': 194, 'feature46': 187, 'feature47': 200, 'feature48': 0, 'feature49': 0, 'feature50': 200, 'feature51': 198, 'feature52': 189, 'feature53': 179, 'feature

In [85]:
# now we have to repeat all procedures mentioned above for test data

test_data = read_csv('/media/anuja/study/Anuja/MS/CS_697A_ML/hw2/optdigits_69.tes', sep=",")
test_data


,0,0.1,0.2,12,13,0.3,0.4,0.5,0.6,0.7,...,0.31,0.32,0.33,1,9,15.2,11.1,3.2,0.34,6
0,0,0,0,8,15,1,0,0,0,0,...,2,0,0,0,7,15,16,11,0,6
1,0,0,1,13,14,3,0,0,0,0,...,0,0,0,3,12,15,14,7,0,6
2,0,0,0,5,14,2,0,0,0,0,...,0,0,0,0,5,16,15,3,0,6
3,0,0,0,10,13,1,0,0,0,1,...,0,0,0,0,7,15,15,5,0,6
4,0,0,0,9,10,0,0,0,0,0,...,0,0,0,0,6,12,14,9,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,0,0,7,16,9,8,2,0,0,5,...,0,0,0,7,15,14,7,0,0,9
356,0,0,5,14,11,8,0,0,0,4,...,0,0,0,7,15,12,5,0,0,9
357,0,0,4,14,15,6,0,0,0,5,...,0,0,0,4,13,15,9,0,0,9
358,0,0,4,10,13,6,0,0,0,1,...,0,0,0,2,14,15,9,0,0,9


In [86]:
# use train_array as numpy array

test_array = np.asarray(test_data)
print(test_array)
[test_rows, test_col] = test_array.shape

# now convert split the arrays into two parts as we have two classes here

test_class6 = test_array[test_array[:,64] == 6, :]
print(test_class6)

test_class9 = test_array[test_array[:,64] == 9, :]
print(test_class9)

[[ 0  0  0 ... 11  0  6]
 [ 0  0  1 ...  7  0  6]
 [ 0  0  0 ...  3  0  6]
 ...
 [ 0  0  4 ...  0  0  9]
 [ 0  0  4 ...  0  0  9]
 [ 0  0  2 ...  0  0  9]]
[[ 0  0  0 ... 11  0  6]
 [ 0  0  1 ...  7  0  6]
 [ 0  0  0 ...  3  0  6]
 ...
 [ 0  0  2 ...  7  0  6]
 [ 0  0  0 ...  5  0  6]
 [ 0  0  2 ...  4  0  6]]
[[ 0  0 11 ...  0  0  9]
 [ 0  0  6 ... 11  1  9]
 [ 0  0  9 ... 11  0  9]
 ...
 [ 0  0  4 ...  0  0  9]
 [ 0  0  4 ...  0  0  9]
 [ 0  0  2 ...  0  0  9]]


In [87]:
# get the occurances or count of both the classes

[test_class6_rows, test_col] = test_class6.shape
print("Number of rows of class 6 of testing dataset : ", test_class6_rows)

[test_class9_rows, test_col] = test_class9.shape
print("Number of rows of class 9 of testing dataset : ", test_class9_rows)



Number of rows of class 6 of testing dataset :  180
Number of rows of class 9 of testing dataset :  180


In [88]:
# now will fine out the class priors

test_prior_class6 = test_class6_rows/test_rows
print("Prior probability of class 6 of testing data : ", test_prior_class6)

test_prior_class9 = test_class9_rows/test_rows
print("Prior probability of class 9 of testing data : ", test_prior_class9)


Prior probability of class 6 of testing data :  0.5
Prior probability of class 9 of testing data :  0.5


In [89]:
# now find the mean and std deviation for each feature of the classes (6 and 9). We have 64 features here and 2 classes.

test_df = pd.DataFrame(columns = ["Test_Mean_class6", "Test_Std_Deviation_class6", "Test_Mean_class9", "Test_Std_Deviation_class9"])

for icol in range(test_col-1):
    
    test_mean_class6 = np.mean(test_class6[:,icol])
    test_std_dv_class6 = np.std(test_class6[:,icol])

    test_mean_class9 = np.mean(test_class9[:,icol])
    test_std_dv_class9 = np.std(test_class9[:,icol])
    
    test_df = test_df.append({'Test_Mean_class6': test_mean_class6, 'Test_Std_Deviation_class6': test_std_dv_class6, 'Test_Mean_class9': test_mean_class9, 'Test_Std_Deviation_class9': test_std_dv_class9}, ignore_index=True)


print(test_df)


    Test_Mean_class6  Test_Std_Deviation_class6  Test_Mean_class9  \
0           0.000000                   0.000000          0.000000   
1           0.000000                   0.000000          0.144444   
2           1.144444                   1.670403          5.683333   
3          11.161111                   3.150245         11.833333   
4           9.566667                   5.250503         11.255556   
..               ...                        ...               ...   
59         10.694444                   2.888862         12.044444   
60         15.094444                   1.381212         13.144444   
61         13.055556                   2.928257          8.894444   
62          4.488889                   3.900839          2.094444   
63          0.094444                   0.523845          0.055556   

    Test_Std_Deviation_class9  
0                    0.000000  
1                    0.460944  
2                    4.025578  
3                    4.244604  
4          

In [101]:
# calculate test errors
test_feature_errors_class6 = []
test_feature_errors_class6_dict = {}

for icol in range(test_col-1):
    test_mean_class6 = test_df.Test_Mean_class6[icol]
    test_std_class6 = test_df.Test_Std_Deviation_class6[icol]
    test_mean_class9 = test_df.Test_Mean_class9[icol]
    test_std_class9 = test_df.Test_Std_Deviation_class9[icol]
    test_error_of_feature = 0
    for irow in range (test_class6_rows):
        if (~(test_std_class6 == 0.0) & ~(test_std_class9 == 0.0)):
            test_g_class6 = -((1/2)* math.log(2*math.pi)) - math.log(test_std_class6) - ((test_class6[irow,icol] - test_mean_class6)** 2)/(2*(test_std_class6**2)) + math.log(test_prior_class6)
            test_g_class9 = -((1/2)* math.log(2*math.pi)) - math.log(test_std_class9) - ((test_class6[irow,icol] - test_mean_class9)** 2)/(2*(test_std_class9**2)) + math.log(test_prior_class6)
            if (test_g_class6 > test_g_class9):
                if ~(test_class6[irow, icol] == 6):
                    test_error_of_feature = test_error_of_feature + 1
            elif (test_g_class6 < test_g_class9):
                if ~(test_class6[irow, icol] == 9):
                    test_error_of_feature = test_error_of_feature + 1
    test_feature_errors_class6_dict.update({"feature" + str(icol+1) :test_error_of_feature })

# for class 9 find error of each feature
for icol in range(test_col-1):
    test_mean_class6 = test_df.Test_Mean_class6[icol]
    test_std_class6 = test_df.Test_Std_Deviation_class6[icol]
    test_mean_class9 = test_df.Test_Mean_class9[icol]
    test_std_class9 = test_df.Test_Std_Deviation_class9[icol]
    test_error_of_feature = 0
    for irow in range (test_class9_rows):
        if (~(test_std_class6 == 0.0) & ~(test_std_class9 == 0.0)):
            test_g_class6 = -((1/2)* math.log(2*math.pi)) - math.log(test_std_class6) - ((test_class6[irow,icol] - test_mean_class6)** 2)/(2*(test_std_class6**2)) + math.log(test_prior_class6)
            test_g_class9 = -((1/2)* math.log(2*math.pi)) - math.log(test_std_class9) - ((test_class9[irow,icol] - test_mean_class9)** 2)/(2*(test_std_class9**2)) + math.log(test_prior_class6)
            if (test_g_class6 > test_g_class9):
                if ~(test_class6[irow, icol] == 6):
                    test_error_of_feature = test_error_of_feature + 1
            elif (test_g_class6 < test_g_class9):
                if ~(test_class6[irow, icol] == 9):
                    test_error_of_feature = test_error_of_feature + 1
            
    test_feature_errors_class6_dict.update({"feature" + str(icol+1) :test_error_of_feature })
    
print("-------------printing list of testing errors for each feature-----------------" + " \n")        
print(test_feature_errors_class6_dict)
print("\n")
print("sorted : ")
test_feature_errors_class6_sortedDict = sorted(test_feature_errors_class6_dict.items(), key=lambda item: item[1], reverse = False)
print(test_feature_errors_class6_sortedDict)

-------------printing list of testing errors for each feature----------------- 

{'feature1': 0, 'feature2': 0, 'feature3': 179, 'feature4': 178, 'feature5': 171, 'feature6': 179, 'feature7': 180, 'feature8': 0, 'feature9': 0, 'feature10': 180, 'feature11': 161, 'feature12': 179, 'feature13': 171, 'feature14': 180, 'feature15': 180, 'feature16': 0, 'feature17': 0, 'feature18': 180, 'feature19': 176, 'feature20': 177, 'feature21': 180, 'feature22': 180, 'feature23': 180, 'feature24': 0, 'feature25': 0, 'feature26': 176, 'feature27': 177, 'feature28': 174, 'feature29': 176, 'feature30': 177, 'feature31': 180, 'feature32': 0, 'feature33': 0, 'feature34': 179, 'feature35': 177, 'feature36': 179, 'feature37': 180, 'feature38': 171, 'feature39': 175, 'feature40': 0, 'feature41': 0, 'feature42': 179, 'feature43': 177, 'feature44': 171, 'feature45': 173, 'feature46': 172, 'feature47': 171, 'feature48': 180, 'feature49': 0, 'feature50': 180, 'feature51': 174, 'feature52': 173, 'feature53': 167,

Answer: For the given testing data set, we get the zero error for 15 features. Thus, following 15 features have the best performance among 64 features. 
feature1, feature2, feature8, feature9, feature16, feature17, feature24, feature25, feature32, feature33, feature40, feature41, feature49, feature57, feature58.



Q2: Use all the 64 features, assume that inputs are 64 dimensional Gaussians, and assume that for each class the covariance matrix is different. Report the training and test confusion matrices and errors. Hint: eliminate features that have variance zero. 

Q3: Repeat Q2, assuming that all the class covariance matrices are the same. 

Q4: Use the first 10 features in Q1 that gave the best test performance and repeat Q2. Compare the test error performance you got to Q2. 